In [1]:
from together import Together
import json
from tqdm import tqdm
import csv
import time

In [5]:
reader = csv.reader(open('IIIT_Dataset.csv', 'r'))
dataset = list(reader)
header = dataset[0]
dataset = dataset[1:]
header

['Image_id', 'Question', 'Answer_1', 'Answer_2', 'Answer_3']

In [6]:
reader = csv.reader(open('urls.csv', 'r'))
urls_ = list(reader)[1:]
urls = {}
for url in urls_:
    urls[url[0]] = url[1]

In [9]:
data = []
for i in tqdm(range(len(dataset))):
    data.append({
        'question': dataset[i][1],
        'answers': [dataset[i][2], dataset[i][3], dataset[i][4]],
        'image_url': urls[dataset[i][0]]
    })

100%|██████████| 24/24 [00:00<00:00, 24391.40it/s]


In [11]:
client = Together(api_key="581f09912b905cd058e92cd71321e5385267b1b3c41396cf8d5057f4ee599bb2")

In [12]:
for i in tqdm(range(len(data))):
    while True:
        try:
            question = data[i]['question']
            prompt = f"You will be asked a question and you have to give a precise answer. If you don't know the answer, respond with 'unanswerable.'\nQuestion: {question}"
            imageUrl = data[i]['image_url']
            output = client.chat.completions.create(
                model="meta-llama/Llama-Vision-Free",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": imageUrl,
                                },
                            },
                        ],
                    }
                ],
                max_tokens=512,
                temperature=0.7,
                top_p=0.7,
                top_k=50,
                repetition_penalty=1,
                stop=["<|eot_id|>","<|eom_id|>"],
                stream=False
            )
            break
        except Exception as e:
            time.sleep(60)
    answer = output.choices[0].message.content
    data[i]['prediction'] = answer
    json.dump(data, open('Llama3_2_IIITD.json', 'w'), indent=4)

  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 24/24 [03:23<00:00,  8.48s/it]
